## 5. Feature Engineering

In [ ]:
def ulez_bool(df):
    """ Function to replace nulls in the 'ULEZ' column. """

    # when the field is not null then use the value
    # in the 'ULEZ' field directly
    if df.ULEZ == df.ULEZ:
        return df.ULEZ
    # when the field is null the use the following cases
    else:
        # when the car is a petrol car
        if      (df.model_year > 2004.00) & \
                ((df.fuel == 'Petrol') |
                                            (df.fuel == 'Petrol Plug-in Hybrid') |
                                            (df.fuel == 'Petrol Hybrid')):
            return 'ULEZ'
        # when the car is a diesel car and registered
        # during or after the second half of 2015
        elif    (df.model_year >= 2015.00 ) & \
                ((df.fuel == 'Diesel') |
                                              (df.fuel == 'Diesel Hybrid') |
                                              (df.fuel == 'Diesel Plug-in Hybrid')) & \
                (df.reg_year_half == 2.00 ):
            return 'ULEZ'
        # when the car is a diesel car and registered
        # before the second half of 2015
        elif    (df.model_year <= 2015.00 ) & \
                ((df.fuel == 'Diesel') |
                                              (df.fuel == 'Diesel Hybrid') |
                                              (df.fuel == 'Diesel Plug-in Hybrid')) & \
                (df.reg_year_half == 1.00 ):
            return 'Not ULEZ'
        elif df.fuel == 'Electric':
            return 'ULEZ'
        else:
            return 'Not ULEZ'

x = cleaned_df[["owners", "reg_year"]]
sel_x = x[["owners","reg_year"]]
df_own = sel_x.groupby("reg_year").agg({"owners":"median"}).reset_index()
dict_own = df_own.to_dict()
ry_list = list(dict_own['reg_year'].values())
ow_list = list(dict_own['owners'].values())
ref_dict = dict(zip(ry_list,ow_list))

def fill_owners(row):
    """ Function to fill nulls in 'owners' column. """
    if row.owners == row.owners:
        return row.owners
    elif row.reg_year == row.reg_year:
        if row.reg_year > 2020:
            return 1
        else:
            return ref_dict[row.reg_year]
    else: return

# full_df['new_owners'] = full_df.apply(fill_owners,axis=1)

In [ ]:
def engineer_features(df):
    df['ULEZ_bool'] = df.apply(ulez_bool,axis=1)
    df['new_owners'] = df.apply(fill_owners,axis=1)
    df['log_mileage'] = np.log(df.mileage)
    df['log_age'] = np.log(df.age)
    df['inv_age'] = df.age ** -1
    df = pd.get_dummies(df, columns = ['ULEZ_bool'], drop_first = True, prefix = 'ulez')
    df = pd.get_dummies(df, columns = ['body'], drop_first = True, prefix = 'body')
    df = pd.get_dummies(df, columns = ['transmission'], drop_first = True, prefix = 'trnm')
    df = pd.get_dummies(df, columns = ['fuel'], drop_first = True, prefix = 'fuel')
    df = pd.get_dummies(df, columns = ['writeoff'], drop_first = True, prefix = 'wo')
    df = pd.get_dummies(df, columns = ['reg_year_half'], drop_first = True, prefix = 'ryh')
    df = pd.get_dummies(df, columns = ['model'], drop_first = True, prefix = 'model')
    df['const'] = 1
    return df

In [ ]:
X_train_eng = engineer_features(X_train)
X_train_eng.head()

In [ ]:
print(X_train_eng.columns)

## 6. Training and Optimising the Linear Regression Model

In [ ]:
# feature_cols = ['mileage', 'owners', 'ULEZ', 'engine', 'reg_year', 'model_year', 'age',
#                 'new_owners', 'log_mileage', 'log_age', 'inv_age', 'ulez_ULEZ',
#                 'body_Coupe', 'body_Estate', 'body_Hatchback', 'body_MPV', 'body_SUV',
#                 'body_Saloon', 'trnm_Manual', 'fuel_Diesel Hybrid',
#                 'fuel_Diesel Plug-in Hybrid', 'fuel_Electric', 'fuel_Petrol',
#                 'fuel_Petrol Hybrid', 'fuel_Petrol Plug-in Hybrid', 'wo_1', 'ryh_2.0',
#                 'model_B Class', 'model_C Class', 'model_CLA Class', 'model_E Class',
#                 'model_GLA Class', 'model_GLC Class', 'model_GLE Class', 'const']

# feature_cols = ['const', 'model_year', 'new_owners', 'log_mileage', 'log_age',
#                 'ulez_ULEZ', 'body_Coupe', 'body_Estate', 'body_Hatchback', 'body_MPV',
#                 'body_SUV', 'body_Saloon', 'trnm_Manual', 'fuel_Diesel Hybrid',
#                 'fuel_Diesel Plug-in Hybrid', 'fuel_Electric', 'fuel_Petrol',
#                 'fuel_Petrol Hybrid', 'fuel_Petrol Plug-in Hybrid', 'wo_1', 'ryh_2.0',
#                 'model_B Class', 'model_C Class', 'model_CLA Class',
#                 'model_E Class', 'model_GLA Class', 'model_GLC Class',
#                 'model_GLE Class']

# feature_cols = ['const', 'model_year', 'new_owners', 'log_mileage', 'log_age',
#                 'ulez_ULEZ', 'body_Coupe', 'body_Estate', 'body_Hatchback', 'body_MPV',
#                 'body_SUV', 'body_Saloon', 'trnm_Manual', 'wo_1', 'ryh_2.0',
#                 'model_B Class', 'model_C Class', 'model_CLA Class',
#                 'model_E Class', 'model_GLA Class', 'model_GLC Class',
#                 'model_GLE Class']

# feature_cols = ['const', 'engine', 'reg_year',
#                 'model_year', 'age', 'new_owners', 'log_mileage', 'log_age',
#                 'ulez_ULEZ', 'trnm_Manual', 'fuel_Diesel Hybrid',
#                 'fuel_Diesel Plug-in Hybrid', 'fuel_Electric', 'fuel_Petrol',
#                 'fuel_Petrol Hybrid', 'fuel_Petrol Plug-in Hybrid', 'wo_1', 'ryh_2.0']

# feature_cols = ['const', 'engine', 'new_owners', 'log_mileage', 'log_age',
#                 'ulez_ULEZ', 'body_Coupe', 'body_Estate',
#                 'body_Saloon', 'trnm_Manual', 'fuel_Diesel Hybrid',
#                 'fuel_Diesel Plug-in Hybrid', 'fuel_Petrol',
#                 'fuel_Petrol Hybrid', 'fuel_Petrol Plug-in Hybrid', 'wo_1', 'ryh_2.0']

feature_cols = ['engine','new_owners', 'log_mileage', 'log_age', 'ulez_ULEZ',
                'body_Coupe', 'body_Estate', 'body_Hatchback',
                'body_Saloon', 'trnm_Manual', 'fuel_Diesel Hybrid',
                'fuel_Diesel Plug-in Hybrid', 'fuel_Petrol',
                'fuel_Petrol Hybrid', 'fuel_Petrol Plug-in Hybrid', 'wo_1', 'ryh_2.0', 'const','model_B Class', 'model_C Class', 'model_CLA Class', 'model_E Class', 'model_GLA Class', 'model_GLC Class','model_GLE Class']

def train_sm_lin_reg(X,y,feature_cols):
    _X = X[feature_cols]
    lin_reg = sm.OLS(y, _X)
    results = lin_reg.fit()
    y_pred = pd.Series(dtype=float)
    y_pred = results.predict(_X)

    rmse = statsmodels.tools.eval_measures.rmse(y, y_pred)

    print(f'rmse: {rmse} \n')
    print(results.params)
    print(results.summary())
    return results

In [ ]:
X_train_eng[feature_cols].shape

In [ ]:
y_train.shape

In [ ]:
first_reg = train_sm_lin_reg(X_train_eng,y_train,feature_cols)
lin_reg = sm.OLS(y_train, X_train_eng[feature_cols])
results = lin_reg.fit()

y_pred = pd.Series()
y_pred = pd.Series(results.predict(X_train_eng))

rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)

print(f'rmse  \t  {rmse}')
print(results.params)
print(results.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def checkVIF(X):
    ''' Function to calculate the VIF of a dataframe '''
    vif = pd.DataFrame() ## Initialising an empty dataframe
    vif['Features'] = feature_cols ## The features are the ones used in training

    ''' Calculating the VIF of each individual feature '''
    vif['VIF'] = [variance_inflation_factor(X[feature_cols].values, i) for i in range(X[feature_cols].shape[1])]
    vif['VIF'] = round(vif['VIF'], 2) ## Rounding it up to 2 decimal places

    vif = vif.sort_values(by = "VIF", ascending = False) ## Sort it from highest to lowest

    return(vif)
checkVIF(X_train_eng)

## 7.Testing the model

In [ ]:
X_test_eng = engineer_features(X_test)
X_test_eng = X_test_eng[feature_cols]


# X_test_eng['y_pred'] = results.predict(X_test_eng)
y_pred_test = first_reg.predict(X_test_eng)
rmse = statsmodels.tools.eval_measures.rmse(y_test, y_pred_test)
print(rmse)

We were not happy with the total RMSE and decided to split the data and run linear regression models using the same approach for every single model of Mercedes in an effort to bring down the RMSE. The result ended up with a mean RMSE across the 8 models of 3065 a decrease 